In [31]:
seqs_train = []
labels_train = []


seqs_test= []
labels_test = []

# Open the file and read the lines
with open("DNA-735-Train.fasta", "r") as f:
    lines = f.readlines()




count = 0
for line in lines:
    if count % 3 == 1:
        seqs_train.append(line.strip())
    elif count % 3 == 2:
        label = line.strip()[1:-1]
        # remove the comma from the label

        label = label.replace(",", "")
        label = ''.join(label.split())
        labels_train.append(label)
    count += 1


# Open the file and read the lines
with open("DNA-180-Test.fasta", "r") as f:
    lines = f.readlines()


count = 0
for line in lines:
    if count % 3 == 1:
        seqs_test.append(line.strip())
    elif count % 3 == 2:
        label = line.strip()[1:-1]
        # remove the comma from the label

        label = label.replace(",", "")
        label = ''.join(label.split())
        labels_test.append(label)
    count += 1

    



In [36]:
import torch
from transformers import T5EncoderModel, T5Tokenizer
import re
import numpy as np
import gc

tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False)
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model = model.eval()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [37]:
def generate_embedding(seq):

    seq = re.sub(r"[UZOB]", "X", seq)  # Replace ambiguous amino acids with X
    seq = " ".join(list(seq))  # Add spaces between each amino acid
    # Ensure the model and tokenizer are on the same device (preferably GPU)
    model.eval()  # Set the model to evaluation mode
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(DEVICE)

    # Tokenize the protein sequence
    inputs = tokenizer(seq, return_tensors="pt")
    inputs = {key: tensor.to(DEVICE) for key, tensor in inputs.items()}

    # Get model outputs
    with torch.no_grad():
        outputs = model(**inputs)

    # outputs.last_hidden_state has shape [1, sequence_length, hidden_size]
    # print("Shape of outputs:", outputs.last_hidden_state.shape)

    # Extract the CLS token embedding (first token of the sequence)
    return outputs.last_hidden_state.cpu().numpy()[0,:-1,:]


from tqdm import tqdm
embeddings=[]
for seq in tqdm(seqs_train):
    embeddings.append(generate_embedding(seq))




100%|██████████| 735/735 [02:05<00:00,  5.85it/s]


In [38]:
embeddings_test=[]
for seq in tqdm(seqs_test):
    embeddings_test.append(generate_embedding(seq))

100%|██████████| 180/180 [01:16<00:00,  2.37it/s]


In [39]:
all_embeddings_train=[]
all_labels_train=[]

for seq_data in embeddings:
    for token_data in seq_data:
        all_embeddings_train.append(token_data)

for seq_data in labels_train:
    for token_data in seq_data:
        all_labels_train.append(int(token_data))




all_embeddings_test=[]
all_labels_test=[]

for seq_data in embeddings_test:
    for token_data in seq_data:
        all_embeddings_test.append(token_data)

for seq_data in labels_test:
    for token_data in seq_data:
        all_labels_test.append(int(token_data))


In [40]:

X_resampled = np.array(all_embeddings_train)
y_resampled = np.array(all_labels_train)




In [41]:
import torch
from torch.nn.utils.rnn import pad_packed_sequence
from torch.nn import Embedding, LSTM,Conv1d,Conv2d,MaxPool2d
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torch.nn as nn
import torch.optim as optim 

import torch.nn.functional as F

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_chanels, **kwargs):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_chanels, **kwargs)
        self.bn = nn.BatchNorm1d(out_chanels)
        
    def forward(self, x):
        return F.relu(self.bn(self.conv(x)))
class InceptionBlock(nn.Module):
    def __init__(
        self, 
        in_channels, 
        out_1x1,
        red_1x1,
        out_3x3,
        out_5x5,
        out_pool,
    ):
        super(InceptionBlock, self).__init__()
        self.branch1 = ConvBlock(in_channels, out_1x1, kernel_size=1) # 
        self.branch2 = nn.Sequential(
            ConvBlock(in_channels, red_1x1, kernel_size=1, padding=0),
            ConvBlock(red_1x1, out_3x3, kernel_size=3, padding=1),
        )
        self.branch3 = nn.Sequential(
            ConvBlock(in_channels, red_1x1, kernel_size=1),
            ConvBlock(red_1x1, out_5x5, kernel_size=5, padding=2),
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, padding=1, stride=1),
            ConvBlock(in_channels, out_pool, kernel_size=1),
        )
    
    def forward(self, x):
        branches = (self.branch1, self.branch2, self.branch3, self.branch4)
        #print((self.branch1(x)).shape)
        return (torch.cat([branch(x) for branch in branches], 1))

class Baseline_1(nn.Module):
    def __init__(
        self, 

       
    ):
        super(Baseline_1, self).__init__()
        self.inception_1=InceptionBlock(1024,128,128,128,128,128)

        self.inception_2=InceptionBlock(512,64,64,64,64,64)

        self.inception_3=InceptionBlock(256,32,32,32,32,32)
        self.linear_1=nn.Linear(128,32)
        self.linear_2=nn.Linear(32,2)
        self.dropout = nn.Dropout(p=0.2)






    
    def forward(self, x):
        x = x.unsqueeze(1)
        x = torch.transpose(x,2,1)
        x=self.inception_1(x)
        x=self.dropout(x)
        x=self.inception_2(x)
        x=self.dropout(x)
        x=self.inception_3(x)
        x=self.dropout(x)
        x=torch.mean(x,dim=2)
        #x = x.reshape(x.shape[0], -1)
        x=self.linear_1(x)
        x=F.relu(self.linear_2(x))
        
        return x


In [42]:
class MinimalDataset(Dataset):
    def __init__(self, data,y) -> None:
        super().__init__()
        self.data = data
        self.y=y

    def __getitem__(self, index):
        # sample={'data':self.data[index],'y':self.y[index]}
        # self.y = int(self.y)
        return [self.data[index],self.y[index]]
       

    def __len__(self):
        return len(self.data)

In [43]:
X_resampled[1].shape

(1024,)

In [44]:
dataset_train= MinimalDataset(X_resampled, y_resampled)
dataset_test= MinimalDataset(all_embeddings_test,all_labels_test)


In [45]:
train_loader =  DataLoader(dataset_train, batch_size= 128, shuffle= True)
test_loader = DataLoader(dataset_test, batch_size = 128,  shuffle = False)

In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Instantiate the model
model = Baseline_1().to(device)





import torch
from torch.utils.data import DataLoader

# Assuming you have a DataLoader for your training data
# and your dataset provides (inputs, labels) tuples

# Count the frequency of each class
class_counts = torch.zeros(2)  # Assuming binary classification; adjust size as needed for more classes
for _, labels in DataLoader(dataset_train, batch_size=1, shuffle=False):
    class_counts += labels.int().bincount(minlength=class_counts.size(0))

# Compute weights as the inverse of the frequency
class_weights = 1. / class_counts
class_weights = class_weights / class_weights.sum()  # Normalize weights

# Move the weights to the correct device
class_weights = class_weights.to(device)
# Create a weighted loss function
criterion = nn.CrossEntropyLoss(class_weights)



# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training Function
def train(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for data, targets in loader:
        data, targets = data.to(device), targets.to(device)

        targets = targets.long()
        
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, targets)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(loader)

from sklearn.metrics import matthews_corrcoef, f1_score, precision_score, roc_auc_score
import numpy as np

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_targets = []
    all_predictions = []
    all_probs = []  # Store probabilities for AUC calculation

    with torch.no_grad():
        for data, targets in loader:
            data, targets = data.to(device), targets.to(device)

            targets = targets.long()
            
            outputs = model(data)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

            # Append to list for MCC, F1, precision, and AUC calculation
            all_targets.extend(targets.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            # Assuming outputs are logits, apply sigmoid to get probabilities
            all_probs.extend(torch.sigmoid(outputs).cpu().numpy()[:, 1])  # Assuming second column has the "positive" class probabilities

    accuracy = 100 * correct / total
    # Calculate MCC
    mcc = matthews_corrcoef(all_targets, all_predictions)
    # Calculate Macro F1 Score
    f1_macro = f1_score(all_targets, all_predictions)
    # Calculate Precision (Weighted)
    precision = precision_score(all_targets, all_predictions, zero_division=0)
    # Calculate AUC
    auc_score = roc_auc_score(all_targets, all_probs)

    return total_loss / len(loader), accuracy, mcc, f1_macro, precision, auc_score

num_epochs = 20

# Include AUC score in your training/testing loop to print along with other metrics:
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    test_loss, test_acc, test_mcc, test_f1_macro, test_precision, test_auc = evaluate(model, test_loader, criterion, device)
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%, Test MCC: {test_mcc:.2f}, Test Macro F1 Score: {test_f1_macro:.2f}, Test Precision: {test_precision:.2f}, Test AUC: {test_auc:.2f}')


Epoch 1/20, Train Loss: 0.3985, Test Loss: 0.3660, Test Accuracy: 85.12%, Test MCC: 0.40, Test Macro F1 Score: 0.41, Test Precision: 0.27, Test AUC: 0.86
Epoch 2/20, Train Loss: 0.3488, Test Loss: 0.3661, Test Accuracy: 85.04%, Test MCC: 0.40, Test Macro F1 Score: 0.40, Test Precision: 0.27, Test AUC: 0.85
Epoch 3/20, Train Loss: 0.3140, Test Loss: 0.3635, Test Accuracy: 86.27%, Test MCC: 0.41, Test Macro F1 Score: 0.42, Test Precision: 0.29, Test AUC: 0.88
Epoch 4/20, Train Loss: 0.2713, Test Loss: 0.3803, Test Accuracy: 89.07%, Test MCC: 0.43, Test Macro F1 Score: 0.45, Test Precision: 0.33, Test AUC: 0.88
Epoch 5/20, Train Loss: 0.2345, Test Loss: 0.5365, Test Accuracy: 90.49%, Test MCC: 0.43, Test Macro F1 Score: 0.46, Test Precision: 0.37, Test AUC: 0.84
Epoch 6/20, Train Loss: 0.2050, Test Loss: 0.4792, Test Accuracy: 90.43%, Test MCC: 0.43, Test Macro F1 Score: 0.46, Test Precision: 0.36, Test AUC: 0.84
Epoch 7/20, Train Loss: 0.1779, Test Loss: 0.5767, Test Accuracy: 91.47%, Te